In [1]:
## Build Data set
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils.data import DataLoader
import torch.nn.functional as F


In [2]:
os.chdir('../')
os.getcwd()

'/Users/markbrackenrig/Documents/2021_FEB/ADSI/assignment_2'

In [3]:
from src.data import make_dataset

In [5]:
data = pd.read_csv('data/raw/beer_reviews.csv')
subsample = data.sample(frac=0.5)

In [6]:
X,y,y_encoder,ohe,scaler = make_dataset.process_data(subsample)

columns dropped
brewery names encoded
scaled data


In [28]:
import pickle as pkl

pkl.dump(ohe, open('models/ohe.pkl', 'wb'))
pkl.dump(y_encoder, open('models/y_encoder.pkl', 'wb'))
pkl.dump(scaler, open('models/scaler.pkl', 'wb'))


## Split into training and test sets

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size = 0.25, random_state = 42)

In [10]:
from src.models.pytorch import PytorchDataset

train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

## Define Baseline Model

In [11]:
from src.models.null import NullModel

baseline_model = NullModel(target_type = 'classification')
y_base = baseline_model.fit_predict(y_train)

In [12]:
from src.models.performance import print_class_perf
print_class_perf(y_base, y_train, set_name='Training', average = 'weighted')

Accuracy Training: 0.07445644067120044
F1 Training: 0.010319192751938586


## Define Model Architecture

Just want to get a basic 1 layer model working before I try and get anything deeper working

In [13]:
from src.models.pytorch import PytorchMultiClass

model = PytorchMultiClass(num_features = X_train.shape[1],class_num= 104)

In [14]:
class PytorchMultiClass(nn.Module):
    def __init__(self, num_features,class_num):
        super(PytorchMultiClass, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 104)
        self.layer_2 = nn.Linear(104, 104)
        self.layer_3 = nn.ReLU()
        self.layer_out = nn.Linear(104,class_num)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.dropout(F.relu(self.layer_1(x)), training=self.training)
        x = self.layer_out(x)
        return self.softmax(x)
    
    
model = PytorchMultiClass(num_features = X_train.shape[1],class_num= 104)

In [15]:
from src.models.pytorch import get_device
device = get_device()
print(device)
model.to(device)

cpu


PytorchMultiClass(
  (layer_1): Linear(in_features=1005, out_features=104, bias=True)
  (layer_2): Linear(in_features=104, out_features=104, bias=True)
  (layer_3): ReLU()
  (layer_out): Linear(in_features=104, out_features=104, bias=True)
  (softmax): Softmax(dim=1)
)

## Train Model

In [16]:
N_EPOCHS = 30
BATCH_SIZE = 10000
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [17]:
from src.models.train_model import train_classification, test_classification

In [18]:

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_acc = test_classification(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\t|\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\t|\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

Epoch: 0: |████████████████████████████████████████████████--| 97.9% Complete
	(train)	|	Loss: 0.0005	|	Acc: 11.2%
	(valid)	|	Loss: 0.0005	|	Acc: 14.4%
Epoch: 1: |████████████████████████████████████████████████--| 97.9% Complete
	(train)	|	Loss: 0.0005	|	Acc: 15.1%
	(valid)	|	Loss: 0.0005	|	Acc: 16.4%
Epoch: 2: |████████████████████████████████████████████████--| 97.9% Complete
	(train)	|	Loss: 0.0005	|	Acc: 16.8%
	(valid)	|	Loss: 0.0005	|	Acc: 17.5%
Epoch: 3: |████████████████████████████████████████████████--| 97.9% Complete
	(train)	|	Loss: 0.0005	|	Acc: 17.3%
	(valid)	|	Loss: 0.0005	|	Acc: 17.9%
Epoch: 4: |████████████████████████████████████████████████--| 97.9% Complete
	(train)	|	Loss: 0.0005	|	Acc: 17.7%
	(valid)	|	Loss: 0.0005	|	Acc: 18.1%
Epoch: 5: |████████████████████████████████████████████████--| 97.9% Complete
	(train)	|	Loss: 0.0005	|	Acc: 18.0%
	(valid)	|	Loss: 0.0005	|	Acc: 18.4%
Epoch: 6: |████████████████████████████████████████████████--| 97.9% Complete
	(train)	|

In [19]:

def predict(test_data, model, criterion, batch_size, device, generate_batch=None):
    """Calculate performance of a Pytorch multi-class classification model

    Parameters
    ----------
    test_data : torch.utils.data.Dataset
        Pytorch dataset
    model: torch.nn.Module
        Pytorch Model
    criterion: function
        Loss function
    bacth_size : int
        Number of observations per batch
    device : str
        Name of the device used for the model
    collate_fn : function
        Function defining required pre-processing steps

    Returns
    -------
    Float
        Loss score
    Float:
        Accuracy Score
    """    
    
    # Set model to evaluation mode
    model.eval()
    test_loss = 0
    test_acc = 0
    
    # Create data loader
    data = DataLoader(test_data, batch_size=batch_size, collate_fn=generate_batch)
    results = []
    # Iterate through data by batch of observations
    for feature, target_class in data:
        
        feature, target_class = feature.to(device), target_class.flatten().to(device)
        
        # Set no update to gradients
        with torch.no_grad():
            # Make predictions
            output = model(feature)
            
        results.append({'prediction': output.argmax(1)[0],'target': target_class[0] })
    return results

In [20]:
test_predictions = predict(test_dataset, model=model, criterion=criterion, batch_size=1, device=device)

In [21]:
test_predictions = pd.DataFrame(test_predictions)

In [22]:
test_predictions.head(4)

,prediction,target
0,tensor(89),tensor(83.)
1,tensor(12),tensor(94.)
2,tensor(9),tensor(9.)
3,tensor(1),tensor(1.)


In [23]:
test_predictions["prediction"][0].int()

tensor(89, dtype=torch.int32)

In [24]:
from sklearn.metrics import confusion_matrix

In [25]:
pd.DataFrame(confusion_matrix(test_predictions['prediction'].astype(int), test_predictions['target'].astype(int))).to_csv('reports/confusion_matrix.csv')

## Save Model

In [27]:
torch.save(model.state_dict(), "models/beer_style_prediction.pt")